In [1]:
import xarray as xr
import numpy as np
import numpy.matlib
import gc
from numba import jit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import mean_squared_error
%matplotlib inline

讀取模式資料

In [2]:
%%time
file = ['lfpw6_1','lfpw7_1', 'cmcc3_1', 'dwd2_1', 'ukmo13_1','ec']
# file = ['lfpw6_1', 'ec', 'ukmo13_1']
system = {}
for f in file:
    ds = xr.open_dataarray(f'D:/share/{f}.nc')
    ds = ds.astype('float64')
    system.setdefault(f,ds)
    del ds

Wall time: 3.81 s


讀取觀測資料

In [3]:
obs = xr.open_dataarray('D:/share/obs_1.nc')
obs = obs.astype('float64')
obs = obs.sel(time=slice('1995-01','2014-12'))

將資料按照leadtime排序,並集合成一個dictionary

In [4]:
%%time
t2m = xr.concat((system['lfpw6_1'],system['lfpw7_1'],system['cmcc3_1'],system['dwd2_1'],system['ukmo13_1'],system['ec']),dim='sys', join='override')
step={}
for leadtime in range(6):
    data = t2m.isel(time=slice(12-leadtime,252-leadtime),step=leadtime)
    step.setdefault(f'{leadtime+1}',data)

Wall time: 1.93 s


將dictonary合併成一個xarray的資料

In [5]:
t2m = xr.concat((step['1'],step['2'],step['3'],step['4'],step['5'],step['6']),dim='step', join='override')

並重新排序xarray的dimensions

In [6]:
t2m = t2m.transpose('sys','time','step','latitude','longitude')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  """Entry point for launching an IPython kernel.


設定dimensions

In [7]:
latitude = t2m.latitude
longitude = t2m.longitude
step = t2m.step 
sys = list(range(len(file)))
time = obs.sel(time=slice('1996-01','2014-12')).time

將模式資料以及觀測資料整理成同樣的維度

In [8]:
t2m = t2m.data
obs = obs.data[np.newaxis,:,np.newaxis,:,:]

In [9]:
print(t2m.shape)
print(obs.shape)

(6, 240, 6, 181, 360)
(1, 240, 1, 181, 360)


先創立一個修正預報值的矩陣

In [10]:
calibrate_data = np.ones((1,228,6,181,360))

In [11]:
gc.collect()

120

---

calibrated by Linear Regression 

In [12]:
%%time

model = LinearRegression()
for i in range(19):  #19
    t2m_traindata =  np.delete(t2m,list(range((i * 12),(12 * i) + 24)),axis=1)
    obs_traindata =  np.delete(obs,list(range((i * 12),(12 * i) + 24)),axis=1)
    
    t2m_testdata = t2m[:,(i * 12):(12 * i) + 24,:]
    obs_testdata = obs[:,(i * 12):(12 * i) + 24,:]
    
    t2m_train_mean = t2m_traindata.mean(axis=1)
    obs_train_mean = obs_traindata.mean(axis=1)
    
    t2m_traindata_anomaly = t2m_traindata - t2m_train_mean[:,np.newaxis,:]
    t2m_traindata_cycmean = t2m_traindata_anomaly.reshape(6,18,12,6,181,360).mean(axis=1)
    t2m_traindata_nomean = t2m_traindata_anomaly - np.tile(t2m_traindata_cycmean,(1,18,1,1,1))
    
    obs_traindata_anomaly = obs_traindata - obs_train_mean[:,np.newaxis,:]
    obs_traindata_cycmean = obs_traindata_anomaly.reshape(1,18,12,1,181,360).mean(axis=1)
    obs_traindata_nomean = obs_traindata_anomaly - np.tile(obs_traindata_cycmean,(1,18,1,1,1))
    
    t2m_test_anomaly = t2m_testdata - t2m_train_mean[:,np.newaxis,:]
    t2m_test_nomean = t2m_test_anomaly - np.tile(t2m_traindata_cycmean,(1,2,1,1,1))
    
    obs_test_anomaly = obs_testdata - obs_train_mean[:,np.newaxis,:]
    obs_test_nomean = obs_test_anomaly - np.tile(obs_traindata_cycmean,(1,2,1,1,1))
    for leadtime in range(6):  #6
        if  i==0 or i==19:
            predictor = obs_traindata_nomean[:, :-1 - leadtime,:]
            predictand = obs_traindata_nomean[:, leadtime + 1:,:]
            t2m_predictand = t2m_traindata_nomean[:, leadtime + 1:,:]

        else:
            before = obs_traindata_nomean[:, 0:(i * 12) - leadtime - 1,:]     #被移除年份之前的觀測值(predictor)
            before2 = obs_traindata_nomean[:, 0 + leadtime + 1:(i * 12),:]    #被移除年份之前的觀測值(predictand)
            t2m_before = t2m_traindata_nomean[:, 0 + leadtime + 1:(i * 12),:] #被移除年份之前的模式值(predictand)
            after = obs_traindata_nomean[:, (i * 12):-leadtime - 1,:]         #被移除年份之後的觀測值(predictor)
            after2 = obs_traindata_nomean[:, (i * 12) + leadtime + 1:,:]      #被移除年份之後的觀測值(predictand)
            t2m_after = t2m_traindata_nomean[:, (i * 12) + leadtime + 1:,:]   #被移除年份之後的模式值(predictand)
            predictor = np.concatenate((before, after), axis=1)     #將觀測值的predictor合併
            predictand = np.concatenate((before2, after2), axis=1)  #將觀測值的predictand合併
            t2m_predictand = np.concatenate((t2m_before, t2m_after), axis=1)
        
        for lat in range(181):   #緯度181
            for lon in range(360):  #經度360
                train = np.concatenate((predictor[:,:,0,lat,lon], t2m_predictand[:,:,leadtime,lat,lon]),axis=0)
                model.fit(train.T, predictand[:,:,0,lat,lon].T)
                
                test = np.concatenate((obs_test_nomean[:,11 - leadtime:-leadtime - 1,0,lat,lon], t2m_test_nomean[:,12:,leadtime,lat,lon]),axis=0)
                obs_nomean_guess = model.predict(test.T)
                pred = obs_nomean_guess + obs_train_mean[:,:,lat,lon] + obs_traindata_cycmean[0,:,:,lat,lon]
                calibrate_data[:,i*12:(i*12)+12,leadtime,lat,lon] = pred.T

Wall time: 1h 11min 38s


---

將修正結果存為Linear2的檔名

In [13]:
np.save('Linear2',calibrate_data)    

將修正結果存為xarray的格式 , 並儲存為NC檔

In [14]:
Linear2 = xr.DataArray(calibrate_data,coords=[[1],time,step,latitude,longitude],dims=['sys','time','step','latitude','longitude'])

In [15]:
time

<xarray.DataArray 'time' (time: 228)>
array(['1996-01-01T00:00:00.000000000', '1996-02-01T00:00:00.000000000',
       '1996-03-01T00:00:00.000000000', ..., '2014-10-01T00:00:00.000000000',
       '2014-11-01T00:00:00.000000000', '2014-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1996-01-01 1996-02-01 ... 2014-12-01
    step        timedelta64[ns] ...
    surface     int64 ...
    valid_time  (time) datetime64[ns] ...
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time

In [16]:
Linear2.to_netcdf('Linear2.nc')

In [17]:
del calibrate_data

In [18]:
gc.collect()

148

---

In [19]:
ds = xr.open_dataarray('Linear2.nc')
ds

<xarray.DataArray (sys: 1, time: 228, step: 6, latitude: 181, longitude: 360)>
[89138880 values with dtype=float64]
Coordinates:
  * sys        (sys) int32 1
  * time       (time) datetime64[ns] 1996-01-01 1996-02-01 ... 2014-12-01
  * step       (step) timedelta64[ns] 31 days 59 days ... 151 days 181 days
  * latitude   (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude  (longitude) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0

In [20]:
ds = ds.mean(dim='sys')   #將ensemble取ensemble mean
ds

<xarray.DataArray (time: 228, step: 6, latitude: 181, longitude: 360)>
array([[[[-28.99498862, -28.99535357, -28.99593385, ..., -28.99590298,
          -28.99739331, -28.99807447],
         [-28.8501869 , -28.84394779, -28.83850078, ..., -28.86758356,
          -28.86223812, -28.85690393],
         [-29.01370285, -28.98775004, -28.97084928, ..., -29.08171723,
          -29.0592671 , -29.0381081 ],
         ...,
         [-23.26289338, -23.29738317, -23.32958598, ..., -23.15042943,
          -23.18402184, -23.21949834],
         [-23.94014924, -23.95612993, -23.97344111, ..., -23.90586863,
          -23.91484795, -23.92633563],
         [-25.27299691, -25.26832733, -25.26624515, ..., -25.26871503,
          -25.26774564, -25.26745172]],

        [[-26.93492198, -26.9371242 , -26.9390573 , ..., -26.92676129,
          -26.92584445, -26.92965674],
         [-26.83242921, -26.82963877, -26.82797699, ..., -26.83973039,
          -26.83913243, -26.83572899],
         [-26.68395741, -26.67892623, -26.68152953, ..., -26.7115849 ,
          -26.71058818, -26.70077692],
         ...,
         [-23.80826506, -23.8372952 , -23.86424942, ..., -23.69721348,
          -23.72736998, -23.7617467 ],
         [-24.48160415, -24.49433913, -24.5082446 , ..., -24.44996036,
          -24.45899083, -24.47000224],
         [-25.94649308, -25.94031003, -25.93565333, ..., -25.95000534,
          -25.94615135, -25.94285705]],

        [[-26.89796484, -26.89987387, -26.9013225 , ..., -26.89713093,
          -26.89883096, -26.90089024],
         [-26.4879781 , -26.48307303, -26.47943885, ..., -26.50949022,
          -26.50300654, -26.49625532],
         [-26.01196607, -26.00199178, -26.00012743, ..., -26.05828235,
          -26.04373871, -26.02767238],
         ...,
         [-23.40331924, -23.42643757, -23.44908484, ..., -23.31473811,
          -23.3381988 , -23.36502366],
         [-24.02591676, -24.04199248, -24.05755322, ..., -23.99723103,
          -24.00748885, -24.01671072],
         [-25.48745085, -25.48746918, -25.4877074 , ..., -25.48257531,
          -25.48407185, -25.48698642]],

        [[-26.0909005 , -26.08938815, -26.08834928, ..., -26.09511889,
          -26.09313746, -26.09278684],
         [-26.04107557, -26.04407756, -26.0473466 , ..., -26.03184199,
          -26.03630971, -26.0402328 ],
         [-25.79899282, -25.80751626, -25.82191344, ..., -25.7653308 ,
          -25.78088556, -25.79309291],
         ...,
         [-23.41301338, -23.43293853, -23.45338721, ..., -23.32641542,
          -23.34970171, -23.37525268],
         [-24.04356621, -24.05197943, -24.06251973, ..., -24.0241709 ,
          -24.0304485 , -24.03666598],
         [-25.47083073, -25.47070267, -25.47218931, ..., -25.47286198,
          -25.47183828, -25.47062184]],

        [[-25.98000787, -25.98114359, -25.9830879 , ..., -25.99634722,
          -25.99684579, -25.99474183],
         [-25.55462021, -25.54242054, -25.53272049, ..., -25.5740031 ,
          -25.56491239, -25.56001935],
         [-25.00822468, -24.98483207, -24.97064999, ..., -25.09242784,
          -25.06265219, -25.03791914],
         ...,
         [-23.3003423 , -23.33502266, -23.36695755, ..., -23.19561845,
          -23.22395884, -23.25747717],
         [-23.94425966, -23.95827699, -23.97431138, ..., -23.91000032,
          -23.92132239, -23.93316223],
         [-25.29167354, -25.28860311, -25.28684387, ..., -25.29792796,
          -25.2946587 , -25.29261791]],

        [[-27.57141728, -27.5686456 , -27.5686381 , ..., -27.58187881,
          -27.57989759, -27.58137165],
         [-27.5519467 , -27.54492235, -27.54152071, ..., -27.56583242,
          -27.56234183, -27.55941172],
         [-27.58063897, -27.57861832, -27.58154943, ..., -27.5662146 ,
          -27.58014414, -27.57974425],
         ...,
         [-24.27977352, -24.30081824, -24.3223607 , ..., -24.18628046,
          -24.21060882, -24.24200928],
         [-24.91990335, -24.931363  , -24.94617817, ..., -24.88665847,
          -24.897862  , 

重新開啟一次觀測資料

In [21]:
obs = xr.open_dataarray('D:/share/obs_1.nc')
obs = obs.astype('float64')

將觀測資料選為跟修正資料相同的時間

In [22]:
obs=obs.sel(time=slice('1996-01','2014-12'))
obs_cycmean = obs.groupby('time.month').mean(dim='time')  #計算觀測資料的年循環(1~12月)
obs

<xarray.DataArray 't2m' (time: 228, latitude: 181, longitude: 360)>
array([[[-28.61230469, -28.61230469, -28.61230469, ..., -28.61230469,
         -28.61230469, -28.61230469],
        [-28.22363281, -28.21582031, -28.20800781, ..., -28.24121094,
         -28.23535156, -28.22949219],
        [-28.21777344, -28.19628906, -28.17871094, ..., -28.27246094,
         -28.25488281, -28.23730469],
        ...,
        [-24.98730469, -25.01660156, -25.04589844, ..., -24.86816406,
         -24.90136719, -24.93847656],
        [-25.67675781, -25.69238281, -25.70996094, ..., -25.63769531,
         -25.64941406, -25.66308594],
        [-27.05957031, -27.05957031, -27.05957031, ..., -27.05957031,
         -27.05957031, -27.05957031]],

       [[-29.63343811, -29.63343811, -29.63343811, ..., -29.63343811,
         -29.63343811, -29.63343811],
        [-29.27015686, -29.27015686, -29.26820374, ..., -29.27796936,
         -29.27601624, -29.27406311],
        [-28.80921936, -28.79554749, -28.78773499, ..., -28.86390686,
         -28.85023499, -28.83460999],
        ...,
        [-32.27992249, -32.33265686, -32.37953186, ..., -32.09242249,
         -32.14710999, -32.20570374],
        [-33.03382874, -33.05335999, -33.07289124, ..., -32.97523499,
         -32.99476624, -33.01429749],
        [-34.60218811, -34.60218811, -34.60218811, ..., -34.60218811,
         -34.60218811, -34.60218811]],

       [[-30.92474365, -30.92474365, -30.92474365, ..., -30.92474365,
         -30.92474365, -30.92474365],
        [-31.29388428, -31.2880249 , -31.28411865, ..., -31.30169678,
         -31.29974365, -31.2958374 ],
        [-31.1317749 , -31.11419678, -31.10247803, ..., -31.1630249 ,
         -31.15325928, -31.14349365],
        ...,
        [-48.82904053, -48.88372803, -48.92669678, ..., -48.64349365,
         -48.6942749 , -48.75091553],
        [-49.54779053, -49.5614624 , -49.57318115, ..., -49.52044678,
         -49.5302124 , -49.53997803],
        [-51.0692749 , -51.0692749 , -51.0692749 , ..., -51.0692749 ,
         -51.0692749 , -51.0692749 ]],

       ...,

       [[-12.45779419, -12.45779419, -12.45779419, ..., -12.45779419,
         -12.45779419, -12.45779419],
        [-12.00662231, -11.99490356, -11.98318481, ..., -12.04177856,
         -12.03005981, -12.01834106],
        [-11.29763794, -11.27224731, -11.24685669, ..., -11.39334106,
         -11.36013794, -11.32888794],
        ...,
        [-47.1882782 , -47.2195282 , -47.24687195, ..., -47.0554657 ,
         -47.09257507, -47.13163757],
        [-47.55351257, -47.56132507, -47.5710907 , ..., -47.5320282 ,
         -47.5398407 , -47.54570007],
        [-48.56913757, -48.56913757, -48.56913757, ..., -48.56913757,
         -48.56913757, -48.56913757]],

       [[-20.85244751, -20.85244751, -20.85244751, ..., -20.85244751,
         -20.85244751, -20.85244751],
        [-22.03213501, -22.01651001, -22.00088501, ..., -22.07901001,
         -22.06338501, -22.04776001],
        [-22.41885376, -22.38565063, -22.35244751, ..., -22.53994751,
         -22.50088501, -22.45986938],
        ...,
        [-32.83486938, -32.85244751, -32.86416626, ..., -32.75674438,
         -32.77627563, -32.79776001],
        [-33.44033813, -33.44815063, -33.45401001, ..., -33.42861938,
         -33.43252563, -33.43643188],
        [-34.51651001, -34.51651001, -34.51651001, ..., -34.51651001,
         -34.51651001, -34.51651001]],

       [[-20.77905273, -20.77905273, -20.77905273, ..., -20.77905273,
         -20.77905273, -20.77905273],
        [-21.24975586, -21.23413086, -21.21655273, ..., -21.29467773,
         -21.28100586, -21.26538086],
        [-20.79272461, -20.75561523, -20.72631836, ..., -20.91186523,
         -20.87280273, -20.83569336],
        ...,
        [-22.71459961, -22.73803711, -22.76147461, ..., -22.62475586,
         -22.65014648, -22.67749023],
        [-23.46655273, -23.48217773, -23.49780273, ..., -23.43530273,
         -23.44506836, -23.45678711],
        [-25.13842773, -25.13842773, -25.13842773, 

---

# 計算CORR 和 RMSE (未減年循環)

將修正值與觀測值轉為numpy陣列做計算

In [23]:
calibrate_data = ds.data
obs1 = obs.data

In [24]:
%%time
cor_coef = np.zeros((6,181,360))
rmse = np.zeros((6,181,360))
for leadtime in range(6):
    for lat in range(181):
        for lon in range(360):
            r, _= stats.pearsonr(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon])     #計算觀測值與修正值的相關係數
            cor_coef[leadtime, lat, lon] = r
            
            r1 = np.sqrt(mean_squared_error(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon]))   #計算觀測值與修正值的RMSE
            std = np.std(obs1[:,lat,lon])    #計算觀測值的標準差
            rmse[leadtime, lat, lon] = r1 / std    #將RMSE標準化

Wall time: 1min 38s


In [25]:
# r1 = np.sqrt(mean_squared_error(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon]))

將相關係數與RMSE轉換為xarray格式資料

In [26]:
corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
rmse = xr.DataArray(rmse,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])

---

In [27]:
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from ipywidgets import interact , fixed

## 相關係數

In [28]:
def myplot_corr(corr,i):
    corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
    cbar_kwargs = {'label': 'correlation coef.','shrink':0.8,'ticks': np.arange(0,1,.1 )}
    # for i in range(6):

            # 創建畫圖空間
    proj = ccrs.PlateCarree() #創建投影
    fig = plt.figure(figsize=(16,9)) #創建頁面
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj}) #子圖

    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                                      linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.xlabels_top = False #關閉頂端標籤
    gl.ylabels_right = False #關閉右側標籤
    gl.xformatter = LONGITUDE_FORMATTER #X軸設為經度格式
    gl.yformatter = LATITUDE_FORMATTER #Y軸設為緯度格式


    corr.isel(step=i).plot(cmap=plt.cm.jet,
                            cbar_kwargs=cbar_kwargs,
                            transform=ccrs.PlateCarree(),
                            vmin=0,
                            vmax=1
                            ) #也可以用Size=?設定圖片大小 robust=True改善離群值
#     plt.imshow

In [29]:
interact(myplot_corr,i=(0,5),corr=fixed(corr))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_corr(corr, i)>

## RMSE

In [30]:
def myplot_rmse(rmse,i):
    cbar_kwargs = {'label': 'RMSE','shrink':0.8,'ticks': np.arange(0,2,.2 )}
#     for i in range(6):

        # 創建畫圖空間
    proj = ccrs.PlateCarree() #創建投影
    fig = plt.figure(figsize=(16,9)) #創建頁面
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj}) #子圖

    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                              linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.xlabels_top = False #關閉頂端標籤
    gl.ylabels_right = False #關閉右側標籤
    gl.xformatter = LONGITUDE_FORMATTER #X軸設為經度格式
    gl.yformatter = LATITUDE_FORMATTER #Y軸設為緯度格式


    rmse.isel(step=i).plot(cmap=plt.cm.jet,
                            cbar_kwargs=cbar_kwargs,
                            transform=ccrs.PlateCarree(),
                            vmin=0,
                            vmax=2
                            ) #也可以用Size=?設定圖片大小 robust=True改善離群值
#         plt.show()

In [31]:
interact(myplot_rmse,i=(0,5),rmse=fixed(rmse)) ;

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

---

# 計算CORR 和 RMSE (減年循環)

計算修正值之年循環(1~12月)

In [32]:
calibrate_data_cycmean = ds.groupby('time.month').mean(dim='time')

In [33]:
calibrate_data_cycmean.shape

(12, 6, 181, 360)

In [34]:
%%time
calibrate_data_no_cycmean = np.zeros((228,6,181,360))
obs_no_cycmean = np.zeros((228,181,360))

calibrate_data_no_cycmean = ds.data - np.tile(calibrate_data_cycmean,(1,19,1,1,1))
obs_no_cycmean = obs.data - np.tile(obs_cycmean.data,(19,1,1))

Wall time: 809 ms


In [35]:
print(calibrate_data_no_cycmean.shape)
print(obs_no_cycmean.shape)

(1, 228, 6, 181, 360)
(228, 181, 360)


In [36]:
%%time
cor_coef = np.zeros((6,181,360))
rmse = np.zeros((6,181,360))
for leadtime in range(6):
    for lat in range(181):
        for lon in range(360):
            r, p= stats.pearsonr(obs_no_cycmean[:,lat,lon], np.squeeze(calibrate_data_no_cycmean)[:,leadtime,lat,lon])
            cor_coef[leadtime, lat, lon] = r
            
            r1 = np.sqrt(mean_squared_error(obs_no_cycmean[:,lat,lon], np.squeeze(calibrate_data_no_cycmean)[:,leadtime,lat,lon]))
            std = np.std(obs_no_cycmean[:,lat,lon])
            rmse[leadtime, lat, lon] = r1 / std

Wall time: 1min 40s


In [37]:
corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
rmse = xr.DataArray(rmse,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])

## 相關係數

In [38]:
interact(myplot_corr,i=(0,5),corr=fixed(corr))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_corr(corr, i)>

## RMSE

In [39]:
interact(myplot_rmse, i=(0,5), rmse=fixed(rmse))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_rmse(rmse, i)>